A closure is a function with an extended scope that encompasses non-global variables referenced in the body of the function but not defined there.

It does not matter whether the function is anonymous or not, what matters is that it can access non-global variables that are defined outside of its body.

In [1]:
class Average:
    """Class to calculate a running average."""
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [2]:
avg = Average()
avg(10)

10.0

In [3]:
avg(11)

10.5

In [4]:
avg(12)

11.0

In [5]:
# Now for a functional implementation


def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [6]:
avg = make_averager()
avg(10)

10.0

In [7]:
avg(11)

10.5

In [8]:
avg(12)

11.0

It is obvious where the avg of the Averager class keeps the history the self.series instance attribute.

The series is a local variable of make_averager because the initialization series = [] happens in the body of that function. When avg(10) is called, make_averager has already returned, its local scope it gone.

Within averager, series is a free variable - meaning a variable
that is not bound in the local scope.

Inspecting the returned averager object we can determine how
python keeps the names of local and free variables.

In [9]:
avg.__code__.co_varnames

('new_value', 'total')

In [10]:
avg.__code__.co_freevars

('series',)

In [11]:
# The binding for series is kept in the __closure__ attribute
# of the returned function
avg.__closure__

(<cell at 0x7f857c1b7288: list object at 0x7f857c1da608>,)

In [12]:
# Each item in av.__closure__ corresponds to a name in avg.__code__.co_freevars
# These items are cells and they have an attribute cell_contents
avg.__closure__[0]

<cell at 0x7f857c1b7288: list object at 0x7f857c1da608>

In [13]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [15]:
type(avg.__closure__)    # Stored in a tuple

tuple

In [16]:
type(avg.__closure__[0])   # Cell objects

cell

In [17]:
type(avg.__closure__[0].cell_contents)

list

In [ ]:
"""
Closure is a function that retains the bindings of the variables that ex
exist when the function is defined, so that they can be used later
when the function is invoked and the defining scope is no longer available.

The only situation in which a function has to deal with external
variables that are non-global is when it is nested in another function.
"""

In [20]:
# Previous implementation wasn't efficient there is no need to
# store all items just the total and count so far


def make_averager():
    """Broken implementation of higher-order function"""
    total = 0
    count = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [21]:
avg = make_averager()

In [24]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [26]:
from dis import dis
dis(avg)

 11           0 LOAD_FAST                1 (count)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_FAST               1 (count)

 12           8 LOAD_FAST                2 (total)
             10 LOAD_FAST                0 (new_value)
             12 INPLACE_ADD
             14 STORE_FAST               2 (total)

 13          16 LOAD_FAST                2 (total)
             18 LOAD_FAST                1 (count)
             20 BINARY_TRUE_DIVIDE
             22 RETURN_VALUE


In [ ]:
# The problem is that the statement += 1 actually means the same
# as count = count + 1 when count is a number or any immutable type
# So we are actually assigning to count in the body of averager and 
# that makes a local variable

In [ ]:
# There was non problem like that before because we never
# assigned to the series object we just called the series.append
# method